In [1]:
from ept_mark import mark
from headers_new import *
import json,os,sys

In [2]:
params = {'output': 'tCl lCl mPk',
          'l_max_scalars': 1000,
          'lensing': 'yes',
          'P_k_max_h/Mpc': 3.,
          'non linear':'halofit', 
          'z_pk': '0.0,1087',
          'A_s': 2.0830e-9,
          'n_s': 0.9649,
          'alpha_s': 0.,
          'h': 0.6736,
          'N_ur': 2.0328,
          'N_ncdm': 1,
          # 'm_ncdm': '0.01,0.05',
          'tau_reio': 0.0544,
          'omega_ncdm': 0.00064420,
          'omega_b': 0.02237,
          'omega_cdm': 0.1200,
          'Omega_k': 0.}
cosmo = Class() 
cosmo.set(params) 
cosmo.compute() 

In [3]:
kmin = 5e-4
kmax = 1.
Nk = 500
h = cosmo.pars['h']
z= 1.1 ###1.
k = np.logspace(np.log10(kmin),np.log10(kmax),Nk) # [h/Mpc]
plin = np.array([cosmo.pk_cb_lin(kk*h,z)*h**3. for kk in k]) 
f = cosmo.scale_independent_growth_factor_f(z)
pars = [1,0,0,0,0,0,0,0,0,0,0]#b1, b2, bs, b3, alpha0, alpha2, alpha4, alpha6, sn, sn2, sn4

In [4]:
def get_pars(path,EPT=True):
    file1 = open(path, 'r')
    Lines = file1.readlines()

    # Strips the newline character
    param_labels = Lines[0].split()
    param_labels = param_labels[1:]
    params = Lines[1].split()
    
    p_names = ['b1','b2','bs','b3','alpha0','alpha2','alpha4','alpha6','SN0','SN2','SN4']
    LPT_params = []
    for pp in p_names:
        if pp in param_labels: LPT_params.append(float(params[param_labels.index(pp)]))
        elif pp=='b1': LPT_params.append(1)
        else: LPT_params.append(0)
    LPT_params[0] -= 1
    if not EPT: return LPT_params
    
    EPT_params = np.zeros(len(p_names))
    EPT_params[-7:] = LPT_params[-7:]
    EPT_params[0] = LPT_params[0] + 1
    EPT_params[1] = LPT_params[1] + 8/21 * LPT_params[0]
    EPT_params[2] = LPT_params[2] - 2/7 * LPT_params[0]
    EPT_params[3] = LPT_params[3] # just leave b3.... not sure what to do with it
    
    return EPT_params


In [5]:
# computes necessary integrals
# this takes pretty long
z_list = [1.1]
mark_list = ['m04']
Rs = [10,20,30]
for mm in mark_list.copy():
    for RR in Rs:
        mark_list.append(mm+'_R%d'%RR)

space_list = ['rsd']
filename = 'mark_dict.json'
with open(filename, "r") as json_file:
    data = json.load(json_file)
stoch = False
if not stoch: stoch_str = ''
else: 
    stoch_str='_stoch'
    # stoch_str='_stoch2'
for zz in z_list:
    for m_name in mark_list:
        Cn = data[m_name]['Cn']
        Rcut = data[m_name]['Rcut']
        print(Cn)
        print(m_name[-3:])
        if Rcut ==15: continue
        # matter 
        pars = get_pars('fits/rsd_matterfit_pkmu.minimum.txt')
        print(pars)
        for ss in space_list:
            gen_name = '%s/z%.3f/%s_matter'%(ss,zz,m_name)
            print(gen_name)
            plin = np.array([cosmo.pk_cb_lin(kk*h,zz)*h**3. for kk in k]) 
            f = cosmo.scale_independent_growth_factor_f(zz)
            temp_mark = mark(k,plin,nk=200,Cn=Cn,R=Rcut,name=gen_name,N=5000,basedir='.')
            # temp_mark.Nskip=Nk
            temp_mark.Nskip=25
            if ss=='real': temp_mark.compute_tables(pars,0)
            elif ss=='rsd': temp_mark.compute_tables(pars,f)
            

            filename = 'output/rsd/z1.100/%s_matter/integral13_unity%s.json'%(m_name[-3:],stoch_str)
            if not os.path.exists(filename):
            # if True:
                integral13_unity = temp_mark.compute_integral13_table(pars,f,unity=True,stoch=stoch)
                dump_dict = {'table':integral13_unity.tolist()}
                with open(filename, 'w') as fp:
                    json.dump(dump_dict, fp)        
            filename = 'output/rsd/z1.100/%s_matter/integral13%s.json'%(m_name[-3:],stoch_str)
            if not os.path.exists(filename):
            # if True:
                integral13 = temp_mark.compute_integral13_table(pars,f,stoch=stoch)
                dump_dict = {'table':integral13.tolist()}
                with open(filename, 'w') as fp:
                    json.dump(dump_dict, fp)

            W_R_int = temp_mark.W_R(temp_mark.kint)
            plin_p = temp_mark.plin_p
                
            filename = 'output/rsd/z1.100/%s_matter/integral22_W%s.json'%(m_name[-3:],stoch_str)
            if not os.path.exists(filename):
            # if True:
            # if True and Rcut!=30 and Rcut !=10 and Rcut !=20:
                integral22_W = temp_mark.compute_integral22_table(pars,f,W_R_int*plin_p,stoch=stoch)
                dump_dict = {'table':integral22_W.tolist()}
                with open(filename, 'w') as fp:
                    json.dump(dump_dict, fp)            
            
        # LRG
        pars = get_pars('fits/rsd_bfit_pkmu.minimum.txt')
        print(pars)
        for ss in space_list:
            # gen_name = '%s/z%.3f/%s_matter'%(ss,zz,m_name)
            gen_name = '%s/z%.3f/%s_LRG'%(ss,zz,m_name)
            print(gen_name)
            plin = np.array([cosmo.pk_cb_lin(kk*h,zz)*h**3. for kk in k]) 
            f = cosmo.scale_independent_growth_factor_f(zz)
            temp_mark = mark(k,plin,nk=200,Cn=Cn,R=Rcut,name=gen_name,N=5000,basedir='.')
            # temp_mark.Nskip=Nk
            temp_mark.Nskip=25
            if ss=='real': temp_mark.compute_tables(pars,0)
            elif ss=='rsd': temp_mark.compute_tables(pars,f)
            filename = 'output/rsd/z1.100/%s_LRG/integral13_unity%s.json'%(m_name[-3:],stoch_str)
            if not os.path.exists(filename):
                integral13_unity = temp_mark.compute_integral13_table(pars,f,unity=True,stoch=stoch)
                dump_dict = {'table':integral13_unity.tolist()}
                with open(filename, 'w') as fp:
                    json.dump(dump_dict, fp)        
            filename = 'output/rsd/z1.100/%s_LRG/integral13%s.json'%(m_name[-3:],stoch_str)
            if not os.path.exists(filename):
                integral13 = temp_mark.compute_integral13_table(pars,f,stoch=stoch)
                dump_dict = {'table':integral13.tolist()}
                with open(filename, 'w') as fp:
                    json.dump(dump_dict, fp)
                
            W_R_int = temp_mark.W_R(temp_mark.kint)
            plin_p = temp_mark.plin_p

            filename = 'output/rsd/z1.100/%s_LRG/integral22_W%s.json'%(m_name[-3:],stoch_str)
            if not os.path.exists(filename):
            # if True and Rcut != 10 and Rcut != 20 and Rcut != 30:
                integral22_W = temp_mark.compute_integral22_table(pars,f,W_R_int*plin_p,stoch=stoch)
                dump_dict = {'table':integral22_W.tolist()}
                with open(filename, 'w') as fp:
                    json.dump(dump_dict, fp)

[0, -1, 0, 0]
m04
[0, -1, 0, 0]
R10
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.8849323e+03  0.0000000e+00]
rsd/z1.100/m04_R10_matter
[ 1.18544270e+00 -1.73435976e+00  4.61319081e-01  0.00000000e+00
 -3.95557780e-01  6.60393440e+00  0.00000000e+00  0.00000000e+00
  3.50860220e+01 -4.71798500e+03  0.00000000e+00]
rsd/z1.100/m04_R10_LRG
[0, -1, 0, 0]
R20
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.8849323e+03  0.0000000e+00]
rsd/z1.100/m04_R20_matter
[ 1.18544270e+00 -1.73435976e+00  4.61319081e-01  0.00000000e+00
 -3.95557780e-01  6.60393440e+00  0.00000000e+00  0.00000000e+00
  3.50860220e+01 -4.71798500e+03  0.00000000e+00]
rsd/z1.100/m04_R20_LRG
[0, -1, 0, 0]
R30
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.000000

In [5]:
def get_Cn_vec(Cn,Cn2=None):
    C0, C1, C2, C3 = Cn
    if Cn2 is None: C0_2, C1_2, C2_2, C3_2 = Cn
    else: C0_2, C1_2, C2_2, C3_2 = Cn2
    return [C0*C0_2, (C0*C1_2+C0_2*C1)/2, (C0* C2_2 +C0_2* C2 )/2, (C0* C3_2 +C0_2* C3 )/2,
            C1*C1_2, (C1*C2_2+C1_2*C2)/2, (C1*C3_2+C1_2*C3)/2, C2*C2_2, (C2*C3_2+C2_2*C3)/2]
    # return [C0**2, C0* C1, C0 *C2, C0 *C3, C1**2, C1 *C2, C1* C3, C2**2, C2*C3]
def combine_mark_params(Cn,table,change=False,Cn2=None):
    if change: 
        Cn_new = [Cn[i]*(-1)**i for i in range(len(Cn))]
    else: Cn_new = Cn    
    if Cn2 is not None: 
        if change: Cn2_new = [Cn2[i]*(-1)**i for i in range(len(Cn2))]
        else: Cn2_new = Cn2
    else: Cn2_new = None
    # print(Cn_new,Cn2_new)
    Cn_vec = get_Cn_vec(Cn_new,Cn2=Cn2_new)
    return np.einsum('i,ijk->jk',Cn_vec,table)


In [9]:
'''
These are blocks reserved for storing content so that the marked classes don't need to be called anywhere else this is for 
MATTER
'''

"\nThese are blocks reserved for storing content so that the marked classes don't need to be called anywhere else this is for \nMATTER\n"

In [6]:
# code to save tables for each radii 

# matter
z_list = [1.1]
mark_list = ['m04']
Rs = [10,20,30]
# Rs = [10,20,30]
for mm in mark_list.copy():
    for RR in Rs:
        mark_list.append(mm+'_R%d'%RR)

space_list = ['rsd']
filename = 'mark_dict.json'
with open(filename, "r") as json_file:
    data = json.load(json_file)

for zz in z_list:
    for m_name in mark_list:
        Cn = data[m_name]['Cn']
        Rcut = data[m_name]['Rcut']
        print(Cn)
        print(m_name[-3:])
        if Rcut ==15: continue
        # matter 
        pars = get_pars('fits/rsd_matterfit_pkmu.minimum.txt')
        print(pars)
        for ss in space_list:
            gen_name = '%s/z%.3f/%s_matter'%(ss,zz,m_name)
            # gen_name = '%s/z%.3f/%s_LRG'%(ss,zz,m_name)
            print(gen_name)
            plin = np.array([cosmo.pk_cb_lin(kk*h,zz)*h**3. for kk in k]) 
            f = cosmo.scale_independent_growth_factor_f(zz)
            temp_mark = mark(k,plin,nk=200,Cn=Cn,R=Rcut,name=gen_name,N=5000,basedir='.')
            if ss=='real': temp_mark.compute_tables(pars,0)
            elif ss=='rsd': temp_mark.compute_tables(pars,f)
                        
            stoch = False
            if stoch: stoch_str = '_stoch'
            else: stoch_str = ''
            M13B_table_new = temp_mark.compute_M13B_table(pars,f,stoch=stoch)
            M13C_table_new = temp_mark.compute_M13C_table(pars,f)
            M22B_table_new = temp_mark.compute_M22B_table(pars,f,stoch=stoch)
            M22C_table_new = temp_mark.compute_M22C_table(pars,f)
            CdCdP_table_new = temp_mark.compute_CdCdP_table(pars,f)
            stoch_SN_table = temp_mark.compute_stoch_SN_table(pars,f,stoch=stoch)
            stoch_BSN_table = temp_mark.compute_stoch_BSN_table(pars,f)
            stoch_dof_table = temp_mark.compute_stoch_dof_table(pars,f)
            
            
            filename = 'output/rsd/z1.100/%s_matter/M_tables%s.json'%(m_name[-3:],stoch_str)
            dump_dict = {'M13B':M13B_table_new.tolist(),
                         'M13C':M13C_table_new.tolist(),
                         'M22B':M22B_table_new.tolist(),
                         'M22C':M22C_table_new.tolist(),
                         'CdCdP':CdCdP_table_new.tolist(),
                         'stoch_SN':stoch_SN_table.tolist(),
                         'stoch_BSN':stoch_BSN_table.tolist(),
                         'stoch_dof':stoch_dof_table.tolist(),
                         }
            with open(filename, 'w') as fp:
                json.dump(dump_dict, fp)

[0, -1, 0, 0]
m04
[0, -1, 0, 0]
R10
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.8849323e+03  0.0000000e+00]
rsd/z1.100/m04_R10_matter
[0, -1, 0, 0]
R20
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.8849323e+03  0.0000000e+00]
rsd/z1.100/m04_R20_matter
[0, -1, 0, 0]
R30
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.8849323e+03  0.0000000e+00]
rsd/z1.100/m04_R30_matter


In [8]:
empty_dict = {'test':'test'}
# reset json file
filename = 'fits/matter_rsd_analytical.json'
# filename = 'fits/matter_rsd_analytical_stoch.json'
# filename = 'fits/rsd_analytical.json'

# Write the empty dictionary to a JSON file
with open(filename, 'w') as json_file:
    json.dump(empty_dict, json_file)


In [9]:
# then code to save each spectra
z_list = [1.1]
mark_list = ['m04']
# Rs = [10,20,30,50,100]
# Rs = [10,20,30,50]
Rs = [10,20,30]
for mm in mark_list.copy():
    for RR in Rs:
        mark_list.append(mm+'_R%d'%RR)

# mark_list = ['m04_R30']

# space_list = ['real','rsd']
space_list = ['rsd']
filename = 'mark_dict.json'
with open(filename, "r") as json_file:
    data = json.load(json_file)

for zz in z_list:
    for m_name in mark_list:
        Cn = data[m_name]['Cn']
        Rcut = data[m_name]['Rcut']
        print(Cn)
        print(m_name[-3:])
        if Rcut ==15: continue
        # matter 
        pars = get_pars('fits/rsd_matterfit_pkmu.minimum.txt')
        print(pars)
        for ss in space_list:
            gen_name = '%s/z%.3f/%s_matter'%(ss,zz,m_name)
            print(gen_name)
            plin = np.array([cosmo.pk_cb_lin(kk*h,zz)*h**3. for kk in k]) 
            f = cosmo.scale_independent_growth_factor_f(zz)
            temp_mark = mark(k,plin,nk=200,Cn=Cn,R=Rcut,name=gen_name,N=5000,basedir='.')
            # temp_mark.Nskip=Nk
            temp_mark.Nskip=40
            if ss=='real': temp_mark.compute_tables(pars,0)
            elif ss=='rsd': temp_mark.compute_tables(pars,f)
            
            stoch = False
            if stoch: stoch_str = '_stoch'
            else: stoch_str = ''
            
            filename = 'output/rsd/z1.100/%s_matter/M_tables%s.json'%(m_name[-3:],stoch_str)
            with open(filename, 'r') as json_file:
                table_data = json.load(json_file)
            M13B_table_new = np.array(table_data['M13B'])
            M13C_table_new = np.array(table_data['M13C'])
            M22B_table_new = np.array(table_data['M22B'])
            M22C_table_new = np.array(table_data['M22C'])
            CdCdP_table_new = np.array(table_data['CdCdP'])
            stoch_SN_table_new = np.array(table_data['stoch_SN'])
            stoch_BSN_table_new = np.array(table_data['stoch_BSN'])
            stoch_dof_table_new = np.array(table_data['stoch_dof'])
            
            for i,key in enumerate(data):
                if key[-3:]!=m_name[-3:]: continue
                Cn = data[key]['Cn']
                for j,key2 in enumerate(data):
                    if i>j:continue
                    if key2[-3:]!=m_name[-3:]: continue
                    if key!=key2 and key[:3]!='m00':continue
                    Cn2 = data[key2]['Cn']
                    # print(key+'_'+key2,Rcut)
                    # input to dict
                    M13_new = combine_mark_params(Cn,M13B_table_new + M13C_table_new,change=True,Cn2=Cn2)
                    M13B_new = combine_mark_params(Cn,M13B_table_new,change=True,Cn2=Cn2)
                    M13C_new = combine_mark_params(Cn,M13C_table_new,change=True,Cn2=Cn2)
                    M22_new = combine_mark_params(Cn,M22B_table_new + M22C_table_new,change=True,Cn2=Cn2)
                    M22B_new = combine_mark_params(Cn,M22B_table_new,change=True,Cn2=Cn2)
                    M22C_new = combine_mark_params(Cn,M22C_table_new,change=True,Cn2=Cn2)
                    CdCdP_new = combine_mark_params(Cn,CdCdP_table_new,change=True,Cn2=Cn2)
                    stoch_SN_new = combine_mark_params(Cn,stoch_SN_table_new,change=True,Cn2=Cn2)
                    stoch_BSN_new = combine_mark_params(Cn,stoch_BSN_table_new,change=True,Cn2=Cn2)
                    stoch_dof_new = combine_mark_params(Cn,stoch_dof_table_new,change=True,Cn2=Cn2)

                    for table in [M13_new,M13B_new,M13C_new,M22_new,M22B_new,M22C_new,stoch_SN_new,stoch_dof_new,stoch_BSN_new]:
                        table[3:] *= 0

                    M13_new = temp_mark.poly2leg(M13_new)
                    M13B_new = temp_mark.poly2leg(M13B_new)
                    M13C_new = temp_mark.poly2leg(M13C_new)
                    M22_new = temp_mark.poly2leg(M22_new)
                    M22B_new = temp_mark.poly2leg(M22B_new)
                    M22C_new = temp_mark.poly2leg(M22C_new)
                    CdCdP_new = temp_mark.poly2leg(CdCdP_new)

                    stoch_BSN_new = temp_mark.poly2leg(stoch_BSN_new) # this shouldn't matter
                    stoch_SN_new = temp_mark.poly2leg(stoch_SN_new)
                    stoch_dof_new = temp_mark.poly2leg(stoch_dof_new)

                    tot_M = 2*M13_new + M22_new + CdCdP_new 
                    tot_M_st = 2*M13_new + M22_new + CdCdP_new + stoch_SN_new


                    # read current dict
                    anal_file = 'fits/matter_rsd_analytical%s.json'%stoch_str
                    with open(anal_file, 'r') as json_file:
                        write_data = json.load(json_file)

                    # The new data to add to the dictionary
                    new_data = {
                        key+'_'+key2: {
                            'k': temp_mark.kv.tolist(),
                            'M_ell': tot_M.tolist(),
                            'Mst_ell': tot_M_st.tolist(),
                            'stoch_BSN_ell': stoch_BSN_new.tolist(),
                            'stoch_SN_ell': stoch_SN_new.tolist(),
                            'stoch_dof_ell': stoch_dof_new.tolist(),
                            'CdCdP_ell': CdCdP_new.tolist(),
                            'M13_ell': M13_new.tolist(),
                            'M13B_ell': M13B_new.tolist(),
                            'M13C_ell': M13C_new.tolist(),
                            'M22_ell': M22_new.tolist(),
                            'M22B_ell': M22B_new.tolist(),
                            'M22C_ell': M22C_new.tolist(),
                            'pars': pars.tolist()
                        }
                    }

                    # Update the existing dictionary with the new data
                    write_data.update(new_data)

                    # Write the updated dictionary back to the JSON file
                    with open(anal_file, 'w') as json_file:
                        json.dump(write_data, json_file, indent=4)


[0, -1, 0, 0]
m04
[0, -1, 0, 0]
R10
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.8849323e+03  0.0000000e+00]
rsd/z1.100/m04_R10_matter
[0, -1, 0, 0]
R20
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.8849323e+03  0.0000000e+00]
rsd/z1.100/m04_R20_matter
[0, -1, 0, 0]
R30
[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  5.0066198e-01 -1.9638121e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.8849323e+03  0.0000000e+00]
rsd/z1.100/m04_R30_matter


In [18]:
# define cov Legendre transform function

def LegendreTrans_Cov(l,p,lp=None,mu_max=1.,Nk=500,Nmu=50):
  '''
  Returns the l'th multipole of |P(k,mu), where P(k,mu) is a 
  vector of length Nk*Nmu. Returns a vector of length Nk.
  '''
  if lp is None: lp = l
  # n = self.Nk ; m = self.Nmu
  n = Nk; m = Nmu
  # mu = self.mu.reshape((n,m))[0]
  mu = np.linspace(0.,1.,50)
  # MU = np.tile(mu,Nk)
  # mu = self.mu.reshape((n,m))[0]
  p_reshaped = p.reshape((n,m))
  result = np.zeros(n)
  for i in range(n):
     integrand = (2*l+1)*(2*lp+1)*p_reshaped[i,:]*scipy.special.legendre(l)(mu)*scipy.special.legendre(lp)(mu)
     result[i] = scipy.integrate.simps(integrand,x=mu)
  return result

def call_markspec(m_name1,m_name2):
    
    simid_list = ['AbacusSummit_base_c000_ph000','AbacusSummit_base_c000_ph001','AbacusSummit_base_c000_ph002','AbacusSummit_base_c000_ph003']
    zz = 1.1
    ss = 'rsd'

    d_pkmu_list = []
    dd_list = []

    for simid in simid_list:
        ff = open('/mocks/results/'+simid+'/z%.3f/%s_pk3d.json'%(zz,ss))
        # ff = open(self.datfn)
        simdata = json.load(ff)
        sim_k = np.array(simdata['k_binc'])
        k = np.array(simdata['k_binc'])
        Nk = len(k)
        Nmu = 50
        mu = np.linspace(0.,1.,Nmu)
        MU = np.tile(mu,Nk)
        dmu = list(mu[1:]-mu[:-1])
        dmu.append(dmu[-1])
        dmu = np.array(dmu)

        dk = list(k[1:]-k[:-1])
        dk.insert(0,dk[0])
        dk = np.array(dk)
        k = np.repeat(k,Nmu)
        dk = np.repeat(dk,Nmu)
        dmu = np.tile(dmu,Nk)

        dd = simdata['matter_%s_matter_%s_ell'%(m_name1,m_name2)]#[:,0]
        dd = np.array(dd)
        dd_list.append(dd)
        dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu),np.repeat(dd[:,2],Nmu)])
        d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]+0.125*(35*MU**4-30*MU**2+3)*dvec[2]
        d_pkmu_list.append(d_pkmu)


    dd_list = np.array(dd_list)
    d_pkmu_list = np.array(d_pkmu_list)
    # print(np.allclose(d_ell_list[0,0],d_ell_list.reshape((len(simid_list),3*Nk))[0,:Nk]))
    dd_avg = np.average(dd_list,axis=0)
    d_pkmu_avg = np.average(d_pkmu_list,axis=0)

    Vsurvey = (2e3)**3

    prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.)
    Cov = prefactor*2*d_pkmu_avg**2
    knl=0.2959184739328827
    kmin = 0.003
    kmax = knl


    k_constraint = (k>kmin)*(k<kmax)
    # Cov += 1e-20
    Cinv = 1/Cov
    Cinv *= k_constraint
    # Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)
    save_dict = {'simid_list':simid_list,'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
                 'd_pkmu':d_pkmu_avg.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd_avg.T.tolist(), 'k':sim_k.tolist()}
    return save_dict

In [11]:
empty_dict = {'test':'test'}
# reset json file
filename = 'fits/matter_rsd_simulation.json'
# filename = 'fits/rsd_analytical.json'

# Write the empty dictionary to a JSON file
with open(filename, 'w') as json_file:
    json.dump(empty_dict, json_file)


In [19]:
# empty_dict = {'test': 0}

# Specify the filename
outfile = 'fits/matter_rsd_simulation.json'

filename = 'mark_dict.json'
with open(filename, "r") as json_file:
    data = json.load(json_file)
for i,key in enumerate(data):
    Cn = data[key]['Cn']
    Rcut = data[key]['Rcut']
    if Rcut ==15 or Rcut ==100: continue
    for j,key2 in enumerate(data):
        if i>j:continue
        if Rcut != data[key2]['Rcut']: continue
        if key!=key2 and key[:3]!='m00':continue
        print(key+'_'+key2,Rcut)
        Cn2 = data[key2]['Cn']
        
        with open(outfile, 'r') as json_file:
            write_data = json.load(json_file)
        try:
            simdata = call_markspec(key,key2)
        except:
            print(key+'_'+key2, 'failed')
            continue
        new_data = {
            key+"_"+key2: simdata
        }
        # Update the existing dictionary with the new data
        write_data.update(new_data)

        # Write the updated dictionary back to the JSON file
        with open(outfile, 'w') as json_file:
            json.dump(write_data, json_file, indent=4)


m00_R10_m00_R10 10
m00_R10_m04_R10 10
m00_R10_m05_R10 10
m00_R10_m06_R10 10
m00_R10_m07_R10 10
m00_R10_m08_R10 10
m00_R10_m09_R10 10
m00_R10_m10_R10 10
m00_R10_m11_R10 10
m00_R10_m14_R10 10
m00_R10_m15_R10 10
m00_R20_m00_R20 20
m00_R20_m04_R20 20
m00_R20_m05_R20 20
m00_R20_m06_R20 20
m00_R20_m07_R20 20
m00_R20_m08_R20 20
m00_R20_m09_R20 20
m00_R20_m10_R20 20
m00_R20_m11_R20 20
m00_R20_m14_R20 20
m00_R20_m15_R20 20
m00_R30_m00_R30 30
m00_R30_m04_R30 30
m00_R30_m05_R30 30
m00_R30_m06_R30 30
m00_R30_m07_R30 30
m00_R30_m08_R30 30
m00_R30_m09_R30 30
m00_R30_m10_R30 30
m00_R30_m11_R30 30
m00_R30_m14_R30 30
m00_R30_m15_R30 30
m00_R50_m00_R50 50
m00_R50_m04_R50 50
m00_R50_m05_R50 50
m00_R50_m06_R50 50
m00_R50_m07_R50 50
m00_R50_m08_R50 50
m00_R50_m09_R50 50
m00_R50_m10_R50 50
m00_R50_m11_R50 50
m00_R50_m14_R50 50
m00_R50_m15_R50 50
m04_R10_m04_R10 10
m04_R20_m04_R20 20
m04_R30_m04_R30 30
m04_R50_m04_R50 50
m05_R10_m05_R10 10
m05_R20_m05_R20 20
m05_R30_m05_R30 30
m05_R50_m05_R50 50
m06_R10_m06_

In [1]:
'''
BIASED TRACERS
'''

'\nBIASED TRACERS\n'

In [10]:
# code to save tables for each radii 

# matter
z_list = [1.1]
mark_list = ['m04']
Rs = [10,20,30]
for mm in mark_list.copy():
    for RR in Rs:
        mark_list.append(mm+'_R%d'%RR)
space_list = ['rsd']
filename = 'mark_dict.json'
with open(filename, "r") as json_file:
    data = json.load(json_file)

for zz in z_list:
    for m_name in mark_list:
        Cn = data[m_name]['Cn']
        Rcut = data[m_name]['Rcut']
        print(Cn)
        print(m_name[-3:])
        if Rcut ==15: continue
        pars = get_pars('fits/rsd_bfit_pkmu.minimum.txt')
        print(pars)
        for ss in space_list:
            gen_name = '%s/z%.3f/%s_LRG'%(ss,zz,m_name)
            print(gen_name)
            plin = np.array([cosmo.pk_cb_lin(kk*h,zz)*h**3. for kk in k]) 
            f = cosmo.scale_independent_growth_factor_f(zz)
            temp_mark = mark(k,plin,nk=200,Cn=Cn,R=Rcut,name=gen_name,N=5000,basedir='.')
            if ss=='real': temp_mark.compute_tables(pars,0)
            elif ss=='rsd': temp_mark.compute_tables(pars,f)
            
            stoch = False
            if stoch: stoch_str = '_stoch'
            else: stoch_str = ''
            M13B_table_new = temp_mark.compute_M13B_table(pars,f,stoch=stoch)
            M13C_table_new = temp_mark.compute_M13C_table(pars,f)
            M22B_table_new = temp_mark.compute_M22B_table(pars,f,stoch=stoch)
            M22C_table_new = temp_mark.compute_M22C_table(pars,f)
            CdCdP_table_new = temp_mark.compute_CdCdP_table(pars,f)
            stoch_SN_table = temp_mark.compute_stoch_SN_table(pars,f,stoch=stoch)
            stoch_BSN_table = temp_mark.compute_stoch_BSN_table(pars,f)
            stoch_dof_table = temp_mark.compute_stoch_dof_table(pars,f)
                        
            filename = 'output/rsd/z1.100/%s_LRG/M_tables%s.json'%(m_name[-3:],stoch_str)

            dump_dict = {'M13B':M13B_table_new.tolist(),
                         'M13C':M13C_table_new.tolist(),
                         'M22B':M22B_table_new.tolist(),
                         'M22C':M22C_table_new.tolist(),
                         'CdCdP':CdCdP_table_new.tolist(),
                         'stoch_BSN':stoch_BSN_table.tolist(),
                         'stoch_SN':stoch_SN_table.tolist(),
                         'stoch_dof':stoch_dof_table.tolist(),
                         }
            with open(filename, 'w') as fp:
                json.dump(dump_dict, fp)

[0, -1, 0, 0]
m04
[0, -1, 0, 0]
R10
[ 1.18544270e+00 -1.73435976e+00  4.61319081e-01  0.00000000e+00
 -3.95557780e-01  6.60393440e+00  0.00000000e+00  0.00000000e+00
  3.50860220e+01 -4.71798500e+03  0.00000000e+00]
rsd/z1.100/m04_R10_LRG
[0, -1, 0, 0]
R20
[ 1.18544270e+00 -1.73435976e+00  4.61319081e-01  0.00000000e+00
 -3.95557780e-01  6.60393440e+00  0.00000000e+00  0.00000000e+00
  3.50860220e+01 -4.71798500e+03  0.00000000e+00]
rsd/z1.100/m04_R20_LRG
[0, -1, 0, 0]
R30
[ 1.18544270e+00 -1.73435976e+00  4.61319081e-01  0.00000000e+00
 -3.95557780e-01  6.60393440e+00  0.00000000e+00  0.00000000e+00
  3.50860220e+01 -4.71798500e+03  0.00000000e+00]
rsd/z1.100/m04_R30_LRG


In [11]:
empty_dict = {'test':'test'}
# reset json file
# filename = 'fits/matter_rsd_analytical.json'
filename = 'fits/rsd_analytical.json'
# filename = 'fits/rsd_analytical_stoch.json'

# Write the empty dictionary to a JSON file
with open(filename, 'w') as json_file:
    json.dump(empty_dict, json_file)


In [12]:
# then code to save each spectra
z_list = [1.1]
mark_list = ['m04']
Rs = [10,20,30]
for mm in mark_list.copy():
    for RR in Rs:
        mark_list.append(mm+'_R%d'%RR)

space_list = ['rsd']
filename = 'mark_dict.json'
with open(filename, "r") as json_file:
    data = json.load(json_file)

print('here')
for zz in z_list:
    for m_name in mark_list:
        Cn = data[m_name]['Cn']
        Rcut = data[m_name]['Rcut']
        print(Cn)
        print(m_name[-3:])
        if Rcut ==15: continue
        # matter 
        pars = get_pars('fits/rsd_bfit_pkmu.minimum.txt')
        print(pars)
        for ss in space_list:
            # gen_name = '%s/z%.3f/%s_matter'%(ss,zz,m_name)
            gen_name = '%s/z%.3f/%s_LRG'%(ss,zz,m_name)
            print(gen_name)
            plin = np.array([cosmo.pk_cb_lin(kk*h,zz)*h**3. for kk in k]) 
            f = cosmo.scale_independent_growth_factor_f(zz)
            temp_mark = mark(k,plin,nk=200,Cn=Cn,R=Rcut,name=gen_name,N=5000,basedir='.')
            if ss=='real': temp_mark.compute_tables(pars,0)
            elif ss=='rsd': temp_mark.compute_tables(pars,f)
            
            stoch = False
            if stoch: stoch_str = '_stoch'
            else: stoch_str = ''

            filename = 'output/rsd/z1.100/%s_LRG/M_tables%s.json'%(m_name[-3:],stoch_str)
            with open(filename, 'r') as json_file:
                table_data = json.load(json_file)
            M13B_table_new = np.array(table_data['M13B'])
            M13C_table_new = np.array(table_data['M13C'])
            M22B_table_new = np.array(table_data['M22B'])
            M22C_table_new = np.array(table_data['M22C'])
            CdCdP_table_new = np.array(table_data['CdCdP'])
            stoch_BSN_table_new = np.array(table_data['stoch_BSN'])
            stoch_SN_table_new = np.array(table_data['stoch_SN'])
            stoch_dof_table_new = np.array(table_data['stoch_dof'])
            
            for i,key in enumerate(data):
                if key[-3:]!=m_name[-3:]: continue
                Cn = data[key]['Cn']
                for j,key2 in enumerate(data):
                    if i>j:continue
                    if key2[-3:]!=m_name[-3:]: continue
                    if key!=key2 and key[:3]!='m00':continue
                    Cn2 = data[key2]['Cn']
                    # print(key+'_'+key2,Rcut)
                    # input to dict
                    M13_new = combine_mark_params(Cn,M13B_table_new + M13C_table_new,change=True,Cn2=Cn2)
                    M13B_new = combine_mark_params(Cn,M13B_table_new,change=True,Cn2=Cn2)
                    M13C_new = combine_mark_params(Cn,M13C_table_new,change=True,Cn2=Cn2)
                    M22_new = combine_mark_params(Cn,M22B_table_new + M22C_table_new,change=True,Cn2=Cn2)
                    M22B_new = combine_mark_params(Cn,M22B_table_new,change=True,Cn2=Cn2)
                    M22C_new = combine_mark_params(Cn,M22C_table_new,change=True,Cn2=Cn2)
                    CdCdP_new = combine_mark_params(Cn,CdCdP_table_new,change=True,Cn2=Cn2)
                    stoch_BSN_new = combine_mark_params(Cn,stoch_BSN_table_new,change=True,Cn2=Cn2)
                    stoch_SN_new = combine_mark_params(Cn,stoch_SN_table_new,change=True,Cn2=Cn2)
                    stoch_dof_new = combine_mark_params(Cn,stoch_dof_table_new,change=True,Cn2=Cn2)

                    for table in [M13_new,M13B_new,M13C_new,M22_new,M22B_new,M22C_new,stoch_SN_new,stoch_BSN_new,stoch_dof_new]:
                        table[3:] *= 0

                    M13_new = temp_mark.poly2leg(M13_new)
                    M13B_new = temp_mark.poly2leg(M13B_new)
                    M13C_new = temp_mark.poly2leg(M13C_new)
                    M22_new = temp_mark.poly2leg(M22_new)
                    M22B_new = temp_mark.poly2leg(M22B_new)
                    M22C_new = temp_mark.poly2leg(M22C_new)
                    CdCdP_new = temp_mark.poly2leg(CdCdP_new)

                    stoch_BSN_new = temp_mark.poly2leg(stoch_BSN_new)
                    stoch_SN_new = temp_mark.poly2leg(stoch_SN_new)
                    stoch_dof_new = temp_mark.poly2leg(stoch_dof_new)

                    tot_M = 2*M13_new + M22_new + CdCdP_new 
                    tot_M_st = 2*M13_new + M22_new + CdCdP_new + stoch_SN_new

                    # read current dict
                    # anal_file = 'fits/matter_rsd_analytical.json'
                    anal_file = 'fits/rsd_analytical%s.json'%stoch_str
                    with open(anal_file, 'r') as json_file:
                        write_data = json.load(json_file)

                    # The new data to add to the dictionary
                    new_data = {
                        key+'_'+key2: {
                            'k': temp_mark.kv.tolist(),
                            'M_ell': tot_M.tolist(),
                            'Mst_ell': tot_M_st.tolist(),
                            'stoch_BSN_ell': stoch_BSN_new.tolist(),
                            'stoch_SN_ell': stoch_SN_new.tolist(),
                            'stoch_dof_ell': stoch_dof_new.tolist(),
                            'CdCdP_ell': CdCdP_new.tolist(),
                            'M13_ell': M13_new.tolist(),
                            'M13B_ell': M13B_new.tolist(),
                            'M13C_ell': M13C_new.tolist(),
                            'M22_ell': M22_new.tolist(),
                            'M22B_ell': M22B_new.tolist(),
                            'M22C_ell': M22C_new.tolist(),
                            'pars': pars.tolist()
                        }
                    }

                    # Update the existing dictionary with the new data
                    write_data.update(new_data)

                    # Write the updated dictionary back to the JSON file
                    with open(anal_file, 'w') as json_file:
                        json.dump(write_data, json_file, indent=4)


here
[0, -1, 0, 0]
m04
[0, -1, 0, 0]
R10
[ 1.18544270e+00 -1.73435976e+00  4.61319081e-01  0.00000000e+00
 -3.95557780e-01  6.60393440e+00  0.00000000e+00  0.00000000e+00
  3.50860220e+01 -4.71798500e+03  0.00000000e+00]
rsd/z1.100/m04_R10_LRG
[0, -1, 0, 0]
R20
[ 1.18544270e+00 -1.73435976e+00  4.61319081e-01  0.00000000e+00
 -3.95557780e-01  6.60393440e+00  0.00000000e+00  0.00000000e+00
  3.50860220e+01 -4.71798500e+03  0.00000000e+00]
rsd/z1.100/m04_R20_LRG
[0, -1, 0, 0]
R30
[ 1.18544270e+00 -1.73435976e+00  4.61319081e-01  0.00000000e+00
 -3.95557780e-01  6.60393440e+00  0.00000000e+00  0.00000000e+00
  3.50860220e+01 -4.71798500e+03  0.00000000e+00]
rsd/z1.100/m04_R30_LRG


In [16]:
# define cov Legendre transform function

def LegendreTrans_Cov(l,p,lp=None,mu_max=1.,Nk=500,Nmu=50):
  '''
  Returns the l'th multipole of |P(k,mu), where P(k,mu) is a 
  vector of length Nk*Nmu. Returns a vector of length Nk.
  '''
  if lp is None: lp = l
  # n = self.Nk ; m = self.Nmu
  n = Nk; m = Nmu
  # mu = self.mu.reshape((n,m))[0]
  mu = np.linspace(0.,1.,50)
  # MU = np.tile(mu,Nk)
  # mu = self.mu.reshape((n,m))[0]
  p_reshaped = p.reshape((n,m))
  result = np.zeros(n)
  for i in range(n):
     integrand = (2*l+1)*(2*lp+1)*p_reshaped[i,:]*scipy.special.legendre(l)(mu)*scipy.special.legendre(lp)(mu)
     result[i] = scipy.integrate.simps(integrand,x=mu)
  return result

def call_markspec(m_name1,m_name2):
    
    simid_list = ['AbacusSummit_base_c000_ph000','AbacusSummit_base_c000_ph001','AbacusSummit_base_c000_ph002','AbacusSummit_base_c000_ph003']
    zz = 1.1
    ss = 'rsd'

    d_pkmu_list = []
    dd_list = []

    for simid in simid_list:
        ff = open('/mocks/results/'+simid+'/z%.3f/%s_pk3d.json'%(zz,ss))
        # ff = open(self.datfn)
        simdata = json.load(ff)
        sim_k = np.array(simdata['k_binc'])
        k = np.array(simdata['k_binc'])
        Nk = len(k)
        Nmu = 50
        mu = np.linspace(0.,1.,Nmu)
        MU = np.tile(mu,Nk)
        dmu = list(mu[1:]-mu[:-1])
        dmu.append(dmu[-1])
        dmu = np.array(dmu)

        dk = list(k[1:]-k[:-1])
        dk.insert(0,dk[0])
        dk = np.array(dk)
        k = np.repeat(k,Nmu)
        dk = np.repeat(dk,Nmu)
        dmu = np.tile(dmu,Nk)

        dd = simdata['LRG_%s_LRG_%s_ell'%(m_name1,m_name2)]#[:,0]
        dd = np.array(dd)
        dd_list.append(dd)
        dvec = np.array([np.repeat(dd[:,0],Nmu),np.repeat(dd[:,1],Nmu),np.repeat(dd[:,2],Nmu)])
        d_pkmu = dvec[0]+0.5*(3*MU**2-1)*dvec[1]+0.125*(35*MU**4-30*MU**2+3)*dvec[2]
        d_pkmu_list.append(d_pkmu)


    dd_list = np.array(dd_list)
    d_pkmu_list = np.array(d_pkmu_list)
    # print(np.allclose(d_ell_list[0,0],d_ell_list.reshape((len(simid_list),3*Nk))[0,:Nk]))
    dd_avg = np.average(dd_list,axis=0)
    d_pkmu_avg = np.average(d_pkmu_list,axis=0)

    Vsurvey = (2e3)**3

    prefactor = (2.*np.pi**2.) / (dk*dmu*Vsurvey*k**2.)
    Cov = prefactor*2*d_pkmu_avg**2
    knl=0.2959184739328827
    kmin = 0.003
    kmax = knl


    k_constraint = (k>kmin)*(k<kmax)
    # Cov += 1e-20
    Cinv = 1/Cov
    Cinv *= k_constraint
    # Cinv = np.einsum('i,j,ij->ij',k_constraint,k_constraint,Cinv)
    save_dict = {'simid_list':simid_list,'z':zz,'space':'rsd','kmin':kmin,'kmax':kmax,'Cov':Cov.tolist(),'Cinv':Cinv.tolist(),
                 'd_pkmu':d_pkmu_avg.tolist(),'k':sim_k.tolist(),'mu':mu.tolist(),'d_ell':dd_avg.T.tolist(), 'k':sim_k.tolist()}
    return save_dict

In [17]:
#### this only runs with access to Abacus output files 

# empty_dict = {'test': 0}

# Specify the filename
outfile = 'fits/rsd_simulation.json'

filename = 'mark_dict.json'
with open(filename, "r") as json_file:
    data = json.load(json_file)
for i,key in enumerate(data):
    Cn = data[key]['Cn']
    Rcut = data[key]['Rcut']
    if Rcut ==15 or Rcut ==100: continue
    for j,key2 in enumerate(data):
        if i>j:continue
        if Rcut != data[key2]['Rcut']: continue
        if key!=key2 and key[:3]!='m00':continue
        print(key+'_'+key2,Rcut)
        Cn2 = data[key2]['Cn']
        
        with open(outfile, 'r') as json_file:
            write_data = json.load(json_file)
        try:
            simdata = call_markspec(key,key2)
        except:
            print(key+'_'+key2, 'failed')
            continue
        new_data = {
            key+"_"+key2: simdata
        }
        # Update the existing dictionary with the new data
        write_data.update(new_data)

        # Write the updated dictionary back to the JSON file
        with open(outfile, 'w') as json_file:
            json.dump(write_data, json_file, indent=4)


m00_R10_m00_R10 10
m00_R10_m04_R10 10
m00_R10_m05_R10 10
m00_R10_m06_R10 10
m00_R10_m07_R10 10
m00_R10_m08_R10 10
m00_R10_m09_R10 10
m00_R10_m10_R10 10
m00_R10_m11_R10 10
m00_R10_m14_R10 10
m00_R10_m15_R10 10
m00_R20_m00_R20 20
m00_R20_m04_R20 20
m00_R20_m05_R20 20
m00_R20_m06_R20 20
m00_R20_m07_R20 20
m00_R20_m08_R20 20
m00_R20_m09_R20 20
m00_R20_m10_R20 20
m00_R20_m11_R20 20
m00_R20_m14_R20 20
m00_R20_m15_R20 20
m00_R30_m00_R30 30
m00_R30_m04_R30 30
m00_R30_m05_R30 30
m00_R30_m06_R30 30
m00_R30_m07_R30 30
m00_R30_m08_R30 30
m00_R30_m09_R30 30
m00_R30_m10_R30 30
m00_R30_m11_R30 30
m00_R30_m14_R30 30
m00_R30_m15_R30 30
m00_R50_m00_R50 50
m00_R50_m04_R50 50
m00_R50_m05_R50 50
m00_R50_m06_R50 50
m00_R50_m07_R50 50
m00_R50_m08_R50 50
m00_R50_m09_R50 50
m00_R50_m10_R50 50
m00_R50_m11_R50 50
m00_R50_m14_R50 50
m00_R50_m15_R50 50
m04_R10_m04_R10 10
m04_R20_m04_R20 20
m04_R30_m04_R30 30
m04_R50_m04_R50 50
m05_R10_m05_R10 10
m05_R20_m05_R20 20
m05_R30_m05_R30 30
m05_R50_m05_R50 50
m06_R10_m06_